In [12]:
# import libraries
import pandas as pd
import numpy as np
import math
import datetime as dt
from datetime import timedelta

# charting libraries
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import s3fs

In [13]:
import boto3
from botocore.config import Config

In [5]:
# s3 = boto3.client('s3')

In [14]:
#################
# Definitions
#################
# Where to read data from
s3_bucket    = 'mfx-sagemaker-dev'

s3_signal_data_key = "demo/XBTUSD_20200729-000000_20200730-140000/2020-07-29T00:00:00.346Z-2020-07-30T14:00:00.961Z-Input_DVCHAIN_top1-1.csv.gz"

# s3_reval_data_key = 'jeff/EURUSD_20200304-040000_20200304-200000/2020-03-04T04:00:00.002Z-2020-03-04T20:00:00.891Z-Input_FASTMATCH-EBSDIRECT-REUTERS-HOTSPOT-GTX-CNX-FXALL_top5-1.csv.gz'
# s3_trigger_data_key =  'go-data/AUDUSD_20200212-060000_20200212-180000/2020-02-12T06:00:00.051Z-2020-02-12T18:00:00.897Z-Input_CITADEL-XTX-GOLDMAN_SACHS_top5-1.csv.gz'

# Bar size
resample_period = '60s'

# Chart settings
chart_padding_secs = 10

# Breakout settings
breakout_sigma = 1.0

# MR settings
mr_sigma = 3

# Hedging level
hedge_level = 3

# Hedger interest to fill time in ms
interest_to_fill = 100
 
# n samples for moving average
ma_samples = 3

# optimise exits
optimise_exit = True

In [15]:
#################
# Load from S3
#################

signal_tick_data = pd.read_csv('s3://{}/{}'.format(s3_bucket, s3_signal_data_key), index_col='t', parse_dates=['t'])
trigger_tick_data = signal_tick_data

reval_tick_data = signal_tick_data
# Source for tick charts
chart_tick_data = reval_tick_data

signal_tick_data['spread'] = signal_tick_data['Offer0'] - signal_tick_data['Bid0']
reval_tick_data['spread'] = reval_tick_data['Offer0'] - reval_tick_data['Bid0']

In [16]:
signal_tick_data.head()

,s,SettlementDate,Bid0Qty,Bid0MinQty,Bid0,Offer0,Offer0Qty,Offer0MinQty,spread
t,,,,,,,,,
2020-07-29 00:00:00.346000+00:00,1,NaN,4.294967e+09,0.0,10927.88,10929.88,4.294967e+09,0.0,2.0
2020-07-29 00:00:00.458000+00:00,2,NaN,4.294967e+09,0.0,10928.94,10930.94,4.294967e+09,0.0,2.0
2020-07-29 00:00:00.562000+00:00,3,NaN,4.294967e+09,0.0,10928.66,10930.66,4.294967e+09,0.0,2.0
2020-07-29 00:00:00.678000+00:00,4,NaN,4.294967e+09,0.0,10928.86,10930.86,4.294967e+09,0.0,2.0
2020-07-29 00:00:00.783000+00:00,5,NaN,4.294967e+09,0.0,10928.96,10930.96,4.294967e+09,0.0,2.0
